In [0]:
df_daily_hal = spark.read.csv("/mnt/vatsal-test/Final/daily_HAL.BSE.csv", header=True, inferSchema=True)

df_daily_infy = spark.read.csv("/mnt/vatsal-test/Final/daily_INFY.BSE.csv", header=True, inferSchema=True)

df_daily_tcs = spark.read.csv("/mnt/vatsal-test/Final/daily_TCS.BSE.csv", header=True, inferSchema=True)

df_monthly_hal = spark.read.csv("/mnt/vatsal-test/Final/monthly_HAL.BSE.csv", header=True, inferSchema=True)
df_monthly_infy = spark.read.csv("/mnt/vatsal-test/Final/monthly_INFY.BSE.csv", header=True, inferSchema=True)
df_monthly_tcs = spark.read.csv("/mnt/vatsal-test/Final/monthly_TCS.BSE.csv", header=True, inferSchema=True)
df_weekly_hal =spark.read.csv("/mnt/vatsal-test/Final/weekly_HAL.BSE.csv", header=True, inferSchema=True)
df_weekly_infy = spark.read.csv("/mnt/vatsal-test/Final/weekly_INFY.BSE.csv", header=True, inferSchema=True)
df_weekly_tcs = spark.read.csv("/mnt/vatsal-test/Final/weekly_TCS.BSE.csv", header=True, inferSchema=True)

In [0]:
from pyspark.sql.functions import col
def clean_data(df):
    return df.withColumn("open", col("open").cast("float")) \
             .withColumn("high", col("high").cast("float")) \
             .withColumn("low", col("low").cast("float")) \
             .withColumn("close", col("close").cast("float")) \
             .withColumn("volume", col("volume").cast("long")) \
             .dropna()

df_daily_hal = clean_data(df_daily_hal)
df_daily_infy= clean_data(df_daily_infy)
df_daily_tcs= clean_data(df_daily_tcs)

In [0]:
def clean_data(df):
    return df.withColumn("open", col("open").cast("float")) \
             .withColumn("high", col("high").cast("float")) \
             .withColumn("low", col("low").cast("float")) \
             .withColumn("close", col("close").cast("float")) \
             .withColumn("volume", col("volume").cast("long")) \
             .dropna()

df_monthly_hal= clean_data(df_monthly_hal)
df_monthly_infy= clean_data(df_monthly_infy)
df_monthly_tcs= clean_data(df_monthly_tcs)

In [0]:
def clean_data(df):
    return df.withColumn("open", col("open").cast("float")) \
             .withColumn("high", col("high").cast("float")) \
             .withColumn("low", col("low").cast("float")) \
             .withColumn("close", col("close").cast("float")) \
             .withColumn("volume", col("volume").cast("long")) \
             .dropna()
df_weekly_hal= clean_data(df_weekly_hal)
df_weekly_infy= clean_data(df_weekly_infy)
df_weekly_tcs= clean_data(df_weekly_tcs)

In [0]:
from pyspark.sql.functions import lit

def add_symbol_column(df, symbol):
    return df.withColumn("symbol", lit(symbol))

df_weekly_hal = add_symbol_column(df_weekly_hal, "HAL")
df_weekly_infy = add_symbol_column(df_weekly_infy, "INFY")
df_weekly_tcs = add_symbol_column(df_weekly_tcs, "TCS")

df_monthly_hal = add_symbol_column(df_monthly_hal, "HAL")
df_monthly_infy = add_symbol_column(df_monthly_infy, "INFY")
df_monthly_tcs = add_symbol_column(df_monthly_tcs, "TCS")

df_daily_hal = add_symbol_column(df_daily_hal, "HAL")
df_daily_infy = add_symbol_column(df_daily_infy, "INFY")
df_daily_tcs = add_symbol_column(df_daily_tcs, "TCS")


In [0]:
f_daily_weekly_monthly_hal = df_daily_hal.join(df_weekly_hal, on=["symbol", "timestamp"], how="inner") \
                                          .join(df_monthly_hal, on=["symbol", "timestamp"], how="inner") \
                                          .select(
                                              df_daily_hal["symbol"],
                                              df_daily_hal["timestamp"],
                                              df_daily_hal["open"].alias("daily_open"),
                                              df_daily_hal["high"].alias("daily_high"),
                                              df_daily_hal["low"].alias("daily_low"),
                                              df_weekly_hal["open"].alias("weekly_open"),
                                              df_weekly_hal["high"].alias("weekly_high"),
                                              df_weekly_hal["low"].alias("weekly_low")
                                          )

df_daily_weekly_monthly_infy = df_daily_infy.join(df_weekly_infy, on=["symbol", "timestamp"], how="inner") \
                                            .join(df_monthly_infy, on=["symbol", "timestamp"], how="inner") \
                                            .select(
                                                df_daily_infy["symbol"],
                                                df_daily_infy["timestamp"],
                                                df_daily_infy["open"].alias("daily_open"),
                                                df_daily_infy["high"].alias("daily_high"),
                                                df_daily_infy["low"].alias("daily_low"),
                                                df_weekly_infy["open"].alias("weekly_open"),
                                                df_weekly_infy["high"].alias("weekly_high"),
                                                df_weekly_infy["low"].alias("weekly_low")
                                            )

df_daily_weekly_monthly_tcs = df_daily_tcs.join(df_weekly_tcs, on=["symbol", "timestamp"], how="inner") \
                                          .join(df_monthly_tcs, on=["symbol", "timestamp"], how="inner") \
                                          .select(
                                              df_daily_tcs["symbol"],
                                              df_daily_tcs["timestamp"],
                                              df_daily_tcs["open"].alias("daily_open"),
                                              df_daily_tcs["high"].alias("daily_high"),
                                              df_daily_tcs["low"].alias("daily_low"),
                                              df_weekly_tcs["open"].alias("weekly_open"),
                                              df_weekly_tcs["high"].alias("weekly_high"),
                                              df_weekly_tcs["low"].alias("weekly_low")
                                          )


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col, avg, to_date, weekofyear, month

def enrich_with_kpis(df):
    window_spec = Window.partitionBy("symbol").orderBy("timestamp")
    df = df.withColumn("daily_price_change_pct", ((col("close") - col("open")) / col("open") * 100)) \
           .withColumn("volume_change", col("volume") - lag("volume").over(window_spec))
    return df
df_daily_hal = enrich_with_kpis(df_daily_hal)
df_daily_infy = enrich_with_kpis(df_daily_infy)
df_daily_tcs = enrich_with_kpis(df_daily_tcs)

#aggregate KPIs over daily, weekly, and monthly periods
def aggregate_kpis(df, period_col):
    return df.groupBy("symbol", period_col).agg(
        avg("daily_price_change_pct").alias(f"avg_{period_col}_price_change_pct"),
        avg("volume_change").alias(f"avg_{period_col}_volume_change")
    )
df_daily_hal = df_daily_hal.withColumn("day", to_date(col("timestamp")))
df_daily_hal = df_daily_hal.withColumn("week", weekofyear(col("timestamp")))
df_daily_hal = df_daily_hal.withColumn("month", month(col("timestamp")))

df_daily_infy = df_daily_infy.withColumn("day", to_date(col("timestamp")))
df_daily_infy = df_daily_infy.withColumn("week", weekofyear(col("timestamp")))
df_daily_infy = df_daily_infy.withColumn("month", month(col("timestamp")))

df_daily_tcs = df_daily_tcs.withColumn("day", to_date(col("timestamp")))
df_daily_tcs = df_daily_tcs.withColumn("week", weekofyear(col("timestamp")))
df_daily_tcs = df_daily_tcs.withColumn("month", month(col("timestamp")))

# Aggregate KPIs over daily, weekly, and monthly periods
df_daily_aggregated_hal = aggregate_kpis(df_daily_hal, "day")
df_weekly_aggregated_hal = aggregate_kpis(df_daily_hal, "week")
df_monthly_aggregated_hal = aggregate_kpis(df_daily_hal, "month")

df_daily_aggregated_infy = aggregate_kpis(df_daily_infy, "day")
df_weekly_aggregated_infy = aggregate_kpis(df_daily_infy, "week")
df_monthly_aggregated_infy = aggregate_kpis(df_daily_infy, "month")

df_daily_aggregated_tcs = aggregate_kpis(df_daily_tcs, "day")
df_weekly_aggregated_tcs = aggregate_kpis(df_daily_tcs, "week")
df_monthly_aggregated_tcs = aggregate_kpis(df_daily_tcs, "month")

# df_daily_aggregated_hal.show()
# df_weekly_aggregated_hal.show()
# df_monthly_aggregated_hal.show()

# df_daily_aggregated_infy.show()
# df_weekly_aggregated_infy.show()
# df_monthly_aggregated_infy.show()

# df_daily_aggregated_tcs.show()
# df_weekly_aggregated_tcs.show()
# df_monthly_aggregated_tcs.show()


In [0]:

# df_daily_aggregated_hal.write.csv("/mnt/vatsal-test/Final/Output_daily_hal_agggg.csv", header=True)
# df_weekly_aggregated_hal.write.csv("/mnt/vatsal-test/Final/Output_weekly_hal_agg.csv", header=True)
# df_monthly_aggregated_hal.write.csv("/mnt/vatsal-test/Final/Output_monthly_hal_agg.csv")

# df_daily_aggregated_infy.write.csv("/mnt/vatsal-test/Final/Output_daily_infy_agg.")
# df_weekly_aggregated_infy.write.csv("/mnt/vatsal-test/Final/Output_weekly_infy_agg.")
# df_monthly_aggregated_infy.write.csv("/mnt/vatsal-test/Final/Output_monthly_infy_agg")

# df_daily_aggregated_tcs.write.csv("/mnt/vatsal-test/Final/Output_daily_tcs_agg.csv")
# df_weekly_aggregated_tcs.write.csv("/mnt/vatsal-test/Final/Output_weekly_tcs_agg.csv")
# df_monthly_aggregated_tcs.write.csv("/mnt/vatsal-test/Final/Output_monthly_tcs_agg.csv")
